#**Pre-request**

##Mount google drive


In [1]:
### **Mount** Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Install pakages


In [2]:
#Install pakages
project_path = "/content/drive/MyDrive/Sem-6/coding/github/fraud_detection/"
%pip install -q -r "{project_path}requirement/Install/eda_requirements.txt" --no-cache-dir


#%pip install -q -r /content/drive/MyDrive/Sem-6/coding/github/fraud_detection/requirements.txt --no-cache-dir
#import os; os._exit(0)   #  restart automatically

######Restart sesstion after this line at first run to make sure that the new libs are used====

In [3]:
project_path = "/content/drive/MyDrive/Sem-6/coding/github/fraud_detection/"
%cd $project_path
%ls $project_path

/content/drive/MyDrive/Sem-6/coding/github/fraud_detection
configs/  notebooks/  requirement/           results/           splits/  tests/
dataset/  README.md   requirements-lock.txt  run_experiment.py  src/


##Import  libs

In [4]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import font_manager as fm
import os
import pandas as pd
import yaml
import logging
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from ydata_profiling import ProfileReport
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
import datetime
import warnings
import re
import sweetviz as sv
import datetime
import datetime
import os


import altair as alt
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from google.colab import data_table
data_table.enable_dataframe_formatter()

# ignore warning related to font
warnings.filterwarnings(
    "ignore",
    message="Glyph .* missing from font",  # Pass the pattern as a STRING
    category=UserWarning
)
## Navigate to project location




#Basic Methods

##Config

In [5]:

def load_config(config_path="configs/baseline.yaml"):
    """Load YAML config file and expand ${root_path} placeholders."""
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)


    # --- Expand ${root_path} placeholders ---
    root = config.get("root_path", "")

    def expand_paths(obj):
        if isinstance(obj, dict):
            return {k: expand_paths(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [expand_paths(i) for i in obj]
        elif isinstance(obj, str) and "${root_path}" in obj:
            return obj.replace("${root_path}", root)
        else:
            return obj

    config = expand_paths(config)
    return config

## CDR dataset

In [6]:
def load_cdr(file_path, nrows=None):
    """Load a CDR CSV file with optional row limit (sample mode)."""
    return pd.read_csv(file_path, nrows=nrows)


def load_all_data(config):
    """
    Load all CSVs defined in config into a dict of DataFrames.
    Uses training.sample_size if available.
    """
    base = config["EDA"]["base_path"]
    files = config["EDA"]["files"]
    sample_size = config.get("EDA", {}).get("sample_size", None)

    data = {}
    for name, fname in files.items():
        path = os.path.join(base, fname)
        df = load_cdr(path, nrows=sample_size)
        data[name] = df
    return data


#EDA

##Profling Methods

###ydata

In [7]:

def ydata_generate_profile(df, output_file="results/profile.html", mode="minimal", overwrite=True):
    """
    Generate profiling report (ydata-profiling) with safe defaults.
    """
    # Drop very high-cardinality or long-text columns in minimal mode
    if mode == "minimal":
       drop_cols = []
       for col in df.columns:
        if df[col].dtype == "object":  # only apply to categorical/text
            if df[col].nunique() > 5000 or df[col].astype(str).str.len().mean() > 100:
                drop_cols.append(col)
        elif df[col].nunique() <= 1:  # constant numeric (useless)
            drop_cols.append(col)

    if drop_cols:
        print(f"Skipping columns for profiling: {drop_cols}")
        df = df.drop(columns=drop_cols)


    # Handle overwrite manually
    if overwrite and os.path.exists(output_file):
        os.remove(output_file)

    profile = ProfileReport(
        df,
        title="Fraud Detection EDA Report",
        explorative=True,
        plot={"wordcloud": False},
        correlations={
            "pearson": {"calculate": mode == "full"},
            "spearman": {"calculate": mode == "full"},
            "kendall": {"calculate": mode == "full"},
            "phi_k": {"calculate": mode == "full"},
            "cramers": {"calculate": mode == "full"},
            "auto": {"calculate": mode == "full"},
        },
        interactions={"continuous": mode == "full"},
        missing_diagrams={"heatmap": mode == "full"},
        duplicates={"calculate": mode == "full"}
    )

    profile.to_file(output_file)  # no overwrite argument
    print(f"Profiling report saved → {output_file}")
    return output_file


###sweetviz

In [8]:
def generate_profile_sweetviz(df, output_file="results/profile_sweetviz.html", target=None):
    """
    Generate a Sweetviz report for df (with optional target column).
    """
    drop_cols = []
    for col in df.columns:
        if df[col].dtype == "object":  # only check high-cardinality categoricals
            if df[col].nunique() > 1000:
                drop_cols.append(col)
        elif df[col].dtype == "datetime64[ns]":  # datetime may break plots
            drop_cols.append(col)

    if drop_cols:
        print(f" Dropping columns for Sweetviz: {drop_cols}")
        df = df.drop(columns=drop_cols)

    report = sv.analyze(df, target_feat=target)
    report.show_html(output_file)
    print(f" Sweetviz report generated at {output_file}")
    return output_file


#Preprocessing Functions

##App

In [9]:
def preprocess_app(df: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocess the APP dataset and prepare for unified timeline merging.
    """
    # --- Basic cleaning ---
    if "busi_name" in df.columns:
        df["busi_name"] = df["busi_name"].fillna("Unknown").astype(str)

    if "flow" in df.columns:
        df["flow"] = df["flow"].fillna(0)
        df["flow"] = df["flow"].clip(lower=0)
        df["flow_norm"] = df["flow"].apply(lambda x: np.log1p(x))

    # --- Month/time processing ---
    if "month_id" in df.columns:
        df["month_id"] = pd.to_datetime(df["month_id"], errors="coerce", format="%Y-%m")

    if "phone_no_m" in df.columns:
        df["month_id"] = df.groupby("phone_no_m")["month_id"].transform(lambda x: x.ffill().bfill())

    if df["month_id"].isna().sum() > 0:
        median_month = df["month_id"].dropna().median()
        df["month_id"] = df["month_id"].fillna(median_month)

    df["month_str"] = df["month_id"].dt.strftime("%Y-%m")

    # --- 🔹 Timeline preparation (moved from build_unified_timeline) ---
    df["event_time"] = df["month_id"] + pd.offsets.MonthEnd(0)
    df["source"] = "APP"

    # Keep only valid entries
    df = df.dropna(subset=["event_time"])

    return df[["phone_no_m", "event_time", "source"] + [col for col in df.columns if col not in ["phone_no_m", "event_time", "source"]]]


##sms

In [10]:
def preprocess_sms(df):
    if "calltype_id" in df.columns:
      df["calltype_id"] = df["calltype_id"].astype("category")
    return df


##user

In [11]:
def preprocess_user(df: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocess the ARPU/User dataset.
    Converts wide monthly ARPU columns (e.g., arpu_201908, arpu_201909, ...)
    into long transactional format suitable for unified timeline integration.
    Retains static user attributes (e.g., city, county, idcard_cnt, label).
    """
    # --- Step 1: Basic cleanup ---
    df["city_name"] = df["city_name"].fillna("Unknown")
    df["county_name"] = df["county_name"].fillna("Unknown")

    if "idcard_cnt" in df.columns and df["idcard_cnt"].isna().sum() > 0:
        df["idcard_cnt"] = df["idcard_cnt"].fillna(df["idcard_cnt"].median())

    # --- Step 2: Detect ARPU columns ---
    arpu_cols = [col for col in df.columns if col.startswith("arpu_")]
    if not arpu_cols:
        raise ValueError("No ARPU columns found (expected columns starting with 'arpu_').")

    # --- Step 3: Melt ARPU columns into long format ---
    id_vars = [c for c in df.columns if c not in arpu_cols]
    df_long = df.melt(
        id_vars=id_vars,
        value_vars=arpu_cols,
        var_name="month_col",
        value_name="arpu_value"
    )

    # --- Step 4: Extract month_id ---
    df_long["month_id"] = (
        df_long["month_col"].str.replace("arpu_", "", regex=False)
        .apply(lambda x: pd.to_datetime(x, format="%Y%m", errors="coerce"))
    )

    # --- Step 5: Clean and normalize ---
    df_long["arpu_value"] = df_long["arpu_value"].fillna(0)
    df_long["arpu_value"] = df_long["arpu_value"].clip(lower=0)

    if "label" in df_long.columns:
        df_long = df_long.dropna(subset=["label"])

    # --- Step 6: Build timeline fields ---
    df_long["event_time"] = df_long["month_id"] + pd.offsets.MonthEnd(0)
    df_long["source"] = "ARPU"

    # --- Step 7: Reorder columns for consistency ---
    ordered_cols = [
        "phone_no_m", "event_time", "source", "month_id",
        "arpu_value", "city_name", "county_name", "idcard_cnt"
    ]
    if "label" in df_long.columns:
        ordered_cols.append("label")

    # Include any remaining columns that might exist but weren’t listed
    remaining_cols = [c for c in df_long.columns if c not in ordered_cols]
    df_long = df_long[ordered_cols + remaining_cols]

    # Drop rows without valid time
    df_long = df_long.dropna(subset=["event_time"])
    df_long = df_long[
        df_long["arpu_value"].notna() & (df_long["arpu_value"] > 0)
    ].reset_index(drop=True)
    return df_long


##Voice

In [12]:
def preprocess_voc(df):
    # Fill categorical columns with 'Unknown' only if missing values exist
    if df["opposite_no_m"].isna().sum() > 0:
        df["opposite_no_m"] = df["opposite_no_m"].fillna("Unknown")
    if df["city_name"].isna().sum() > 0:
        df["city_name"] = df["city_name"].fillna("Unknown")
    if df["county_name"].isna().sum() > 0:
        df["county_name"] = df["county_name"].fillna("Unknown")
    if df["imei_m"].isna().sum() > 0:
        df["imei_m"] = df["imei_m"].fillna("Unknown")

    # calltype_id → EDA shows no missing, so don’t overwrite unnecessarily

    # Drop rows missing datetime
    if df["start_datetime"].isna().sum() > 0:
        df = df.dropna(subset=["start_datetime"])

    # Handle call duration
    if df["call_dur"].isna().sum() > 0:
        df["call_dur"] = df["call_dur"].fillna(0)



    return df


#Load data

In [15]:
config = load_config("configs/baseline.yaml")
print(config)



{'root_path': '/content/drive/MyDrive/Sem-6/coding/github/fraud_detection/', 'sample_data': {'size': 1000, 'fraud_ratio': 0.3, 'full_data': '/content/drive/MyDrive/Sem-6/coding/github/fraud_detection/dataset/CallChinses/raw/full', 'save_path': '/content/drive/MyDrive/Sem-6/coding/github/fraud_detection/dataset/CallChinses/raw/sample/sampled', 'files': {'app': 'app.csv', 'sms': 'sms.csv', 'user': 'user.csv', 'voc': 'voc.csv'}}, 'clustring': {'raw': {'base_path': '/content/drive/MyDrive/Sem-6/coding/github/fraud_detection/dataset/CallChinses/raw/full/', 'files': {'app': 'app.csv', 'sms': 'sms.csv', 'user': 'user.csv', 'voc': 'voc.csv'}}}, 'EDA': {'base_path': '/content/drive/MyDrive/Sem-6/coding/github/fraud_detection/dataset/CallChinses/raw/full/', 'files': {'app': 'app.csv', 'sms': 'sms.csv', 'user': 'user.csv', 'voc': 'voc.csv'}, 'sample_size': None, 'preprocessing_output': '/content/drive/MyDrive/Sem-6/coding/github/fraud_detection/dataset/CallChinses/preprocessed/full/'}, 'Agg': {'b

In [16]:
# Load raw data
data = load_all_data(config)



/tmp/ipython-input-2393918373.py:3: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path, nrows=nrows)



# Excute EDA
# ===============================


In [17]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
for name, df in data.items():

    # -------------------------------
    # Load dataset-specific preprocessing config
    # -------------------------------
    preprocess_cfg = config["preprocessing"].get(name, {})

    numeric_features = preprocess_cfg.get("numeric_features", [])
    categorical_features = preprocess_cfg.get("categorical_features", [])
    datetime_features = preprocess_cfg.get("datetime_features", [])
    target = preprocess_cfg.get("target", None)

    print(f"\nPreprocessing {name}")
    print(f"   numeric: {numeric_features}")
    print(f"   categorical: {categorical_features}")
    print(f"   datetime: {datetime_features}")
    print(f"   target: {target}")

    # -------------------------------
    # Profiling with ydata_profiling
    # -------------------------------
    profiling_mode = config["training"].get("profiling_mode", "minimal")
    ydata_output = f"results/profile_{name}_{timestamp}.html"
    ydata_generate_profile(df, output_file=ydata_output, mode=profiling_mode)

    # -------------------------------
    # Profiling with Sweetviz
    # -------------------------------
    sweetviz_output = f"results/profile_{name}_sweetviz_{timestamp}.html"
    generate_profile_sweetviz(df, output_file=sweetviz_output, target=target)




Preprocessing app
   numeric: ['flow', 'month_id']
   categorical: ['busi_name']
   datetime: []
   target: None
Skipping columns for profiling: ['phone_no_m']


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 3/3 [00:14<00:00,  4.82s/it]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

...

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Profiling report saved → results/profile_voc_20251025_095923.html
 Dropping columns for Sweetviz: ['phone_no_m', 'opposite_no_m', 'start_datetime', 'imei_m']


                                             |          | [  0%]   00:00 -> (? left)

Report results/profile_voc_sweetviz_20251025_095923.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
 Sweetviz report generated at results/profile_voc_sweetviz_20251025_095923.html


#Preprocessing

In [18]:
output_dir = config["EDA"]["preprocessing_output"]


timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
for name, df in data.items():
  if name == "app":
      print(f"\nPreprocessing {name}")
      df = preprocess_app(df)
      output_path = os.path.join(output_dir, f"preprocessed_{name}_{timestamp}.csv")
      df.to_csv(output_path, index=False)
      display(df.head())
      print(f"✅ Saved {name} → {output_path}")

  elif name == "sms":
      print(f"\nPreprocessing {name}")
      df = preprocess_sms(df)
      output_path = os.path.join(output_dir, f"preprocessed_{name}_{timestamp}.csv")
      df.to_csv(output_path, index=False)
      display(df.head())
      print(f"✅ Saved {name} → {output_path}")

  elif name == "user":
    print(f"\nPreprocessing {name}")
    df = preprocess_user(df)
    output_path = os.path.join(output_dir, f"preprocessed_{name}_{timestamp}.csv")
    df.to_csv(output_path, index=False)
    display(df.head())
    print(f"✅ Saved {name} → {output_path}")

  elif name == "voc":
    print(f"\nPreprocessing {name}")
    df = preprocess_voc(df)
    output_path = os.path.join(output_dir, f"preprocessed_{name}_{timestamp}.csv")
    df.to_csv(output_path, index=False)
    display(df.head())
    print(f"✅ Saved {name} → {output_path}")





Preprocessing app


,phone_no_m,event_time,source,busi_name,flow,month_id,flow_norm,month_str
0,416cec0f25b93f08bfd9cff44382c5da3a6346beb16a2c...,2019-11-30,APP,Unknown,0.000000,2019-11-01,0.000000,2019-11
1,26fcb7c6f4125ee5445756d4ff5346c29f2aff7d0f2e31...,2019-11-30,APP,Unknown,0.000000,2019-11-01,0.000000,2019-11
2,1a58c82eeefdb00ea6abf2e6010a8e808b27e1802b764d...,2019-12-31,APP,Unknown,0.021529,2019-12-01,0.021301,2019-12
3,1a58c82eeefdb00ea6abf2e6010a8e808b27e1802b764d...,2019-12-31,APP,微信,0.001846,2019-12-01,0.001845,2019-12
4,1a58c82eeefdb00ea6abf2e6010a8e808b27e1802b764d...,2019-12-31,APP,高德导航,0.002288,2019-12-01,0.002285,2019-12


✅ Saved app → /content/drive/MyDrive/Sem-6/coding/github/fraud_detection/dataset/CallChinses/preprocessed/full/preprocessed_app_20251025_100144.csv

Preprocessing sms


,phone_no_m,opposite_no_m,calltype_id,request_datetime
0,0251387744988114430181c3e680a3733001a26fe686bd...,df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f...,2,2019-08-01 12:13:08
1,0251387744988114430181c3e680a3733001a26fe686bd...,df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f...,2,2019-08-01 12:13:08
2,0251387744988114430181c3e680a3733001a26fe686bd...,df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f...,2,2019-08-01 12:13:08
3,296cfae1d838070c4dd05a125a85c3d29bbb95f713c2ea...,1205bb229d750a6bcb3f9c33893b5d68c8fc8a6443b0a7...,2,2019-08-01 17:46:44
4,39de6ef3a87b8e660e42496450c54b731f3621ca708944...,972affd4be9e7596420379b7e7910843759cdfbd81315e...,2,2019-08-01 16:46:30


✅ Saved sms → /content/drive/MyDrive/Sem-6/coding/github/fraud_detection/dataset/CallChinses/preprocessed/full/preprocessed_sms_20251025_100144.csv

Preprocessing user


,phone_no_m,event_time,source,month_id,arpu_value,city_name,county_name,idcard_cnt,label,month_col
0,672ddbf02a5544d32e4ecc9433b1981bffe23bf912273a...,2019-08-31,ARPU,2019-08-01,46.06,绵阳,江油分公司,1,0,arpu_201908
1,5e1272273e041e82cb275ae877710be98cdaf5b0a8f34d...,2019-08-31,ARPU,2019-08-01,79.00,德阳,旌阳分公司,1,0,arpu_201908
2,eaab3472ec87b076e69e6e8bb62b14341638fc63661a6c...,2019-08-31,ARPU,2019-08-01,18.85,成都,金堂分公司,2,0,arpu_201908
3,0ce1bb415704178bf44e9c9b431a39b083a132c8e6d99f...,2019-08-31,ARPU,2019-08-01,19.20,成都,高新分公司,2,0,arpu_201908
4,28b87f35f63f65096a53e3a4c97eaffd4a6c43ffa7e92d...,2019-08-31,ARPU,2019-08-01,50.00,德阳,旌阳分公司,1,0,arpu_201908


✅ Saved user → /content/drive/MyDrive/Sem-6/coding/github/fraud_detection/dataset/CallChinses/preprocessed/full/preprocessed_user_20251025_100144.csv

Preprocessing voc


,phone_no_m,opposite_no_m,calltype_id,start_datetime,call_dur,city_name,county_name,imei_m
0,3b6bef554a4754fa9977b50d99e6777c9fb26e585961a4...,3572f19c52faeea5111ddd79dfc3643171d00e6d8b3fd9...,2,2019-10-17 17:16:25,13,Unknown,Unknown,d2940d7cde5092d809af2e4bb8f6936a405382bd541542...
1,3b6bef554a4754fa9977b50d99e6777c9fb26e585961a4...,3572f19c52faeea5111ddd79dfc3643171d00e6d8b3fd9...,2,2019-10-17 17:36:20,16,Unknown,Unknown,d2940d7cde5092d809af2e4bb8f6936a405382bd541542...
2,ecd1b3f5a38576ef9872d1cb7a762f31869153de736e7a...,c386d11d5203136e924e070afcac734cdaea6691472e84...,1,2019-08-28 09:46:50,18,Unknown,Unknown,071e8dcb4010c643c48339cb8abf4a69e03b37afade98d...
3,41a97bdfe77f3210deba78689baecb110922878be2f14c...,79ef199a532bcb9a89990d0996cee3d40557f06b9e5265...,1,2019-09-18 16:19:20,34,Unknown,Unknown,5f054945a5037e688abe0ef36d88a0fd351d476e3320c2...
4,41a97bdfe77f3210deba78689baecb110922878be2f14c...,e273542129cf7fa37c28d3f4c95b7b9e8bcb561bbabc50...,2,2019-09-18 16:18:46,23,Unknown,Unknown,5f054945a5037e688abe0ef36d88a0fd351d476e3320c2...


✅ Saved voc → /content/drive/MyDrive/Sem-6/coding/github/fraud_detection/dataset/CallChinses/preprocessed/full/preprocessed_voc_20251025_100144.csv


#Freez

In [ ]:
%pip freeze > "{project_path}requirement/freez/eda_requirements-lock.txt"
